In [1]:
import pandas as pd
import sqlite3
import re

## Load Dataset

In [2]:
# Load dataset

df = pd.read_csv("train_preprocess.tsv.txt" ,encoding="latin1",sep='\t',header=None,names=["text","label"])

In [3]:
df.head()

,text,label
0,warung ini dimiliki oleh pengusaha pabrik tahu...,positive
1,mohon ulama lurus dan k212 mmbri hujjah partai...,neutral
2,lokasi strategis di jalan sumatera bandung . t...,positive
3,betapa bahagia nya diri ini saat unboxing pake...,positive
4,duh . jadi mahasiswa jangan sombong dong . kas...,negative


In [4]:
df.shape

(11000, 2)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11000 entries, 0 to 10999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    11000 non-null  object
 1   label   11000 non-null  object
dtypes: object(2)
memory usage: 172.0+ KB


- Dataset mengandung 11000 row and 2 columns (text and label)
- Semua tipe datanyas object (string)

In [6]:
df.isnull().sum()

text     0
label    0
dtype: int64

In [7]:
#Cek duplikasi data

df.duplicated().sum()

67

In [8]:
# Hapus duplicated data

df = df.drop_duplicates()

In [9]:
print('Jumlah Duplikasi Data = {}'.format(df.duplicated().sum()))
print('Duplikasi Data Tela dihapus...')

Jumlah Duplikasi Data = 0
Duplikasi Data Tela dihapus...


In [10]:
#cek lagi
df.shape

(10933, 2)

In [11]:
df.head()

,text,label
0,warung ini dimiliki oleh pengusaha pabrik tahu...,positive
1,mohon ulama lurus dan k212 mmbri hujjah partai...,neutral
2,lokasi strategis di jalan sumatera bandung . t...,positive
3,betapa bahagia nya diri ini saat unboxing pake...,positive
4,duh . jadi mahasiswa jangan sombong dong . kas...,negative


- Ada 67 data yang diduplikasi dalam dataset dan sudah dihapus, Update terdiri dari 10933 baris.

## Text Processing

In [12]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
daftar_alay = pd.read_csv('new_kamusalay.csv', encoding='ISO-8859-1', header=None)
daftar_alay = daftar_alay.rename(columns={0: 'awal', 1: 'ganti'}) 

In [14]:
daftar_alay

,awal,ganti
0,anakjakartaasikasik,anak jakarta asyik asyik
1,pakcikdahtua,pak cik sudah tua
2,pakcikmudalagi,pak cik muda lagi
3,t3tapjokowi,tetap jokowi
4,3x,tiga kali
...,...,...
15162,mendikbud,menteri pendidikan dan kebudayaan
15163,mendag,menteri perdagangan
15164,menaker,menteri tenaga kerja
15165,memetwit,mentwit


In [15]:
#Cleansing
def bersih_bersih(text):
    text = re.sub('\n',' ',text)
    text = re.sub('rt',' ',text)
    text = re.sub('user',' ',text)
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))',' ',text)
    text = re.sub('  +', ' ', text)
    text = re.sub('[^0-9a-zA-Z]+', ' ', text)
    return text

def lowercase(text):
    return text.lower()

kamus_alay = dict(zip(daftar_alay['awal'], daftar_alay['ganti']))
def ganti_alay(text):
    return ' '.join([kamus_alay[word] if word in kamus_alay else word for word in text.split(' ')])

stop_words_indonesia = set(stopwords.words('indonesian'))
def menghilangkan_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words_indonesia]
    filtered_text = ' '.join(filtered_words)
    return filtered_text

#Jadikan satu fungsi Cleansing
def cleansing(text):
    text = bersih_bersih(text) 
    text = lowercase(text)
    text = ganti_alay(text)
    text = menghilangkan_stopwords(text)
    return text

In [18]:
# Apply cleansing
df['text'] = df['text'].apply(cleansing)

In [19]:
df.head()

,text,label
0,warung dimiliki pengusaha pabrik puluhan terke...,positive
1,mohon ulama lurus k212 mmbri hujjah ai diwlh s...,neutral
2,lokasi strategis jalan sumatra bandung nya nya...,positive
3,betapa bahagia nya unboxing paket barang nya b...,positive
4,aduh mahasiswa sombong kasih kakak kuning bela...,negative


In [28]:
# Membuat koneksi ke database SQLite
conn = sqlite3.connect('database.db')

In [29]:
# Menyimpan DataFrame ke dalam tabel 'tabel_cleansed' dalam database
df.to_sql('tabel_cleansed', con=conn, index=False, if_exists='replace')

10933

In [30]:
print("DataFrame yang telah di-cleansing berhasil disimpan ke dalam tabel 'tabel_cleansed' di database.")

DataFrame yang telah di-cleansing berhasil disimpan ke dalam tabel 'tabel_cleansed' di database.


In [31]:
# Close the database

conn.commit()
conn.close()